<a href="https://colab.research.google.com/github/dbigman/project-dsml-interactive-travel-planner/blob/main/Functions_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import requests
from datetime import datetime, timedelta
import json
import chromadb
from sentence_transformers import SentenceTransformer


# Functions

In [ ]:
API_KEY = "f5b3be307a91026889509d3e0cbee098"  # Replace with your API key
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"

def find_weather_forecast(date, location):
    """
    Retrieves the weather forecast for a given date and location using OpenWeather API.

    Parameters:
    - date (str): Target date in 'YYYY-MM-DD' format.
    - location (str): City name or "city, country" (e.g., "San Juan, PR").

    Returns:
    - dict: Weather forecast details (temperature, description, etc.).
    """
    try:
        # Get weather data
        params = {
            "q": location,
            "appid": API_KEY,
            "units": "metric",  # Use "imperial" for Fahrenheit
        }
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        if response.status_code != 200:
            return {"error": data.get("message", "Failed to fetch weather data")}

        # Convert input date to datetime
        target_date = datetime.strptime(date, "%Y-%m-%d")

        # Find the closest forecast for the given date
        closest_forecast = None
        min_diff = timedelta.max

        for forecast in data["list"]:
            forecast_time = datetime.utcfromtimestamp(forecast["dt"])
            time_diff = abs(forecast_time - target_date)

            if time_diff < min_diff:
                min_diff = time_diff
                closest_forecast = forecast

        if closest_forecast:
            return {
                "date": closest_forecast["dt_txt"],
                "temperature": closest_forecast["main"]["temp"],
                "description": closest_forecast["weather"][0]["description"],
                "humidity": closest_forecast["main"]["humidity"],
                "wind_speed": closest_forecast["wind"]["speed"],
            }
        else:
            return {"error": "No forecast found for the specified date"}

    except Exception as e:
        return {"error": str(e)}

# Example usage:
print(find_weather_forecast("2025-02-15", "San Juan, PR"))

{'date': '2025-02-15 00:00:00', 'temperature': 23.52, 'description': 'clear sky', 'humidity': 79, 'wind_speed': 5.95}


## Landmark Embeddings

In [ ]:
# Load embedding model
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Define ChromaDB storage path (Google Drive in your case)
CHROMA_DB_PATH = "/content/drive/MyDrive/IronHack_final_project/chromadb"

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)

# Create or get collection for landmarks
landmark_collection = client.get_or_create_collection(name="landmarks")

# Load landmarks data from JSON file
with open("/content/drive/MyDrive/IronHack_final_project/landmarks.json", "r", encoding="utf-8") as file:
    landmarks = json.load(file)

# Store landmarks in ChromaDB
for landmark in landmarks:
    # Ensure required fields exist
    if "name" not in landmark or "description" not in landmark:
        print(f"Skipping entry due to missing fields: {landmark}")
        continue

    # Generate a unique ID from the landmark name
    landmark_id = landmark["name"].replace(" ", "_").lower()

    # Convert description from list to string (if needed)
    if isinstance(landmark["description"], list):
        landmark["description"] = " ".join(landmark["description"]).replace("\\n", " ").strip()

    # Generate embedding for the landmark description
    embedding = embedding_model.encode(landmark["description"]).tolist()

    # Ensure all metadata values are valid (convert None to "Unknown")
    metadata = {
        "name": landmark["name"] if landmark["name"] is not None else "Unknown",
        "description": landmark["description"] if landmark["description"] is not None else "Unknown",
        "category": landmark.get("category", "Unknown") if landmark.get("category") is not None else "Unknown",
        "municipality": landmark.get("municipality", "Unknown") if landmark.get("municipality") is not None else "Unknown",
        "coordinates": str(landmark["coordinates"]) if landmark["coordinates"] is not None else "Unknown",
        "source_file": landmark.get("source_file", "Unknown") if landmark.get("source_file") is not None else "Unknown"
    }

    # Add to ChromaDB
    landmark_collection.add(
        ids=[landmark_id],
        embeddings=[embedding],
        metadatas=[metadata]
    )

print("Landmarks stored in ChromaDB successfully!")

Landmarks stored in ChromaDB successfully!


## Municipalities Embeddings

In [ ]:
import json
import chromadb
from sentence_transformers import SentenceTransformer

# Load embedding model
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Define ChromaDB storage path (Google Drive in your case)
CHROMA_DB_PATH = "/content/drive/MyDrive/IronHack_final_project/chromadb_municipalities"

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)

# Create or get collection for municipalities
municipality_collection = client.get_or_create_collection(name="municipalities")

# Load municipalities data from JSON file
with open("/content/drive/MyDrive/IronHack_final_project/municipalities.json", "r", encoding="utf-8") as file:
    municipalities = json.load(file)

# Store municipalities in ChromaDB
for municipality in municipalities:
    # Ensure required fields exist
    if "name" not in municipality or "description" not in municipality:
        print(f"Skipping entry due to missing fields: {municipality}")
        continue

    # Generate a unique ID from the municipality name
    municipality_id = municipality["name"].replace(" ", "_").lower()

    # Convert description from list to string (if needed)
    if isinstance(municipality["description"], list):
        municipality["description"] = " ".join(municipality["description"]).replace("\\n", " ").strip()

    # Generate embedding for the municipality description
    embedding = embedding_model.encode(municipality["description"]).tolist()

    # Ensure all metadata values are valid (convert None to "Unknown")
    metadata = {
        "name": municipality["name"] if municipality["name"] is not None else "Unknown",
        "description": municipality["description"] if municipality["description"] is not None else "Unknown",
        "category": municipality.get("category", "Municipality") if municipality.get("category") is not None else "Municipality",
        "coordinates": str(municipality["coordinates"]) if municipality["coordinates"] is not None else "Unknown",
        "source_file": municipality.get("source_file", "Unknown") if municipality.get("source_file") is not None else "Unknown"
    }

    # Add to ChromaDB
    municipality_collection.add(
        ids=[municipality_id],
        embeddings=[embedding],
        metadatas=[metadata]
    )

print("Municipalities stored in ChromaDB successfully!")

Municipalities stored in ChromaDB successfully!


## Appropriate location function

In [ ]:
def rank_appropriate_locations(user_prompt, top_k=5):
    """
    Finds and ranks appropriate landmarks based on user input.

    Args:
        user_prompt (str): The user's interest (e.g., "I love the beach and history").
        top_k (int): Number of top results to return.

    Returns:
        List of top_k ranked locations.
    """
    # Ensure ChromaDB is initialized
    global landmark_collection  # Ensure we're using the same collection

    # Convert user prompt into an embedding
    user_embedding = embedding_model.encode(user_prompt).tolist()

    # Retrieve top relevant locations using similarity search
    search_results = landmark_collection.query(
        query_embeddings=[user_embedding],
        n_results=top_k
    )

    # Extract matched locations
    ranked_locations = [
        {
            "name": metadata["name"],
            "description": metadata["description"],
            "category": metadata["category"],
            "municipality": metadata["municipality"],
            "score": score  # Similarity score
        }
        for metadata, score in zip(search_results["metadatas"][0], search_results["distances"][0])
    ]

    return ranked_locations

## Apropriate location test

In [ ]:
user_prompt = "I love beaches and historical places"
top_recommendations = rank_appropriate_locations(user_prompt)

for idx, place in enumerate(top_recommendations, 1):
    print(f"{idx}. {place['name']} ({place['category']}) - {place['municipality']}")
    print(f"   Score: {place['score']:.4f}")
    print(f"   {place['description']}\n")

1. Caracas Beach (Vieques) - Wikipedia (Landmark) - Unknown
   Score: 44.3121
   Caracas Beach(Spanish:Playa Caracas), also known asRed BeachinEnglish, is a beach on the southern coast ofViequesin the barrio ofPuerto Ferro. The beach is famous for its white sand, its clear blue waters,tidal poolsand shallow cove perfect for snorkeling. It used to be considered remote but it is now easily reachable through a paved road. The beach has also been developed for visitors with picnic areas, bathrooms and a large parking area. It is located in and managed by theVieques National Wildlife Refugeand there is also a trail that leads to a nearby hill that offers beautiful views of the beach and cove.[1][2][3]  18631N652447W﻿ / ﻿18.10861N 65.41306W﻿ /18.10861; -65.41306

2. Esperanza Beach - Wikipedia (Landmark) - Unknown
   Score: 49.4853
   Esperanza Beach(Spanish:Playa La Esperanza) is a popular beach on the southern coast ofViequesinLa Esperanza,Puerto Real. In comparison to other beaches in the

## Find info on location

In [ ]:
def find_info_on_location(user_prompt, location, top_k=3):
    """
    Retrieves relevant information about a given location based on user query.

    Args:
        user_prompt (str): The user's specific request (e.g., "Tell me about the history of Old San Juan").
        location (str): The location for which the user wants information.
        top_k (int): Number of top results to return.

    Returns:
        List of relevant document excerpts.
    """
    # Ensure ChromaDB is initialized
    global landmark_collection

    # Encode user query into an embedding
    query_embedding = embedding_model.encode(f"{user_prompt} about {location}").tolist()

    # Search for relevant information in ChromaDB
    search_results = landmark_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    # Extract matched documents
    relevant_info = [
        {
            "name": metadata["name"],
            "description": metadata["description"],
            "category": metadata["category"],
            "municipality": metadata["municipality"],
            "score": score  # Similarity score
        }
        for metadata, score in zip(search_results["metadatas"][0], search_results["distances"][0])
        if location.lower() in metadata["name"].lower()  # Filter by location name
    ]

    return relevant_info

## Find info on location test

In [ ]:
user_query = "What is the history of this place?"
location = "Caguas"

info = find_info_on_location(user_query, location)

for idx, entry in enumerate(info, 1):
    print(f"{idx}. {entry['name']} ({entry['category']}) - {entry['municipality']}")
    print(f"   Score: {entry['score']:.4f}")
    print(f"   {entry['description']}\n")


1. Caguas barrio-pueblo - Wikipedia (Landmark) - Unknown
   Score: 30.0830
   Caguas Pueblois abarrioanddowntownarea that serves the administrative center (seat) of the city and municipality ofCaguas, a municipality of Puerto Rico. It is bordered by theCagitas Riverto the north and located two miles southwest of theRo Grande de Loza. Its population in 2020 was 19,020.[4][5][6][7]  As was customary inSpain, in Puerto Rico, the municipality has a barrio calledpueblo(barrio-pueblostarting with the 1990 US Census) which contains a central plaza or main town square, the municipal buildings (such as the city hall), and a Catholic church.Fiestas patronales(patron saint festivals) are held in the main town square every year.[8][9]The municipal and mayoral offices were located in theCity Hall buildingat the western edge of the main town square until 2010 when a new City Hall building located in the northwestern side of the downtown area was inaugurated.  Thehistoric downtown district(pueblo) of

# Assembling the assistant

In [ ]:
# Imports
import openai

In [ ]:
api_key_path = "/content/drive/MyDrive/IronHack_final_project/API_Key1.txt"

# Read the key from the file
with open(api_key_path, "r") as file:
    openai.api_key = file.read().strip()  # Strip removes any extra spaces or newlines

print("API Key Loaded Successfully!")  # Just to confirm it's working

API Key Loaded Successfully!


In [ ]:
# Function to query the LLM
def query_llm(prompt):
    response = openai.Completion.create(
        model="gpt-4",  # You can also use "gpt-3.5-turbo"
        prompt=prompt,
        max_tokens=150,
        temperature=0.7,
    )
    return response.choices[0].text.strip()

In [ ]:
def generate_assistant_prompt(user_query, location=None):
    # If the user mentions a location, adjust the prompt to provide more details
    if location:
        return f"User asked about '{location}'. The location is part of a large list of landmarks and municipalities in Puerto Rico. Answer the user's query related to this location: '{user_query}'."
    else:
        return f"User has the following query: '{user_query}'. Provide suggestions about relevant locations or landmarks in Puerto Rico based on the user’s interests."

def find_relevant_documents(user_query):
    # Query ChromaDB for relevant documents based on user interests or location
    query_embedding = embedding_model.encode(user_query).tolist()

    # Retrieve matching documents from the ChromaDB collections
    results = landmark_collection.query(
        query_embeddings=[query_embedding],
        n_results=3  # Number of results to retrieve
    )

    print("Results from ChromaDB:", results)  # Add this line to inspect the results
    return results

def assistant_response(user_query):
    # Step 1: Find relevant documents based on user query
    relevant_docs = find_relevant_documents(user_query)

    # Step 2: Check if relevant documents exist and are not None
    if relevant_docs and relevant_docs.get("documents") and relevant_docs["documents"][0] is not None:
        # Extract the first document and get the metadata
        location_info = relevant_docs["metadatas"][0].get("name", "No location found")
        prompt = generate_assistant_prompt(user_query, location=location_info)
    else:
        prompt = f"Sorry, I couldn't find any matching places for '{user_query}'. Could you provide more specific information?"

    # Step 3: Query the LLM for an answer
    response = query_llm(prompt)

    return response

## Testing the LLM


In [ ]:
# Simulate a conversation with the assistant
user_input = "I love the sun and beaches. What places should I visit in Puerto Rico?"
assistant_output = assistant_response(user_input)

print(f"Assistant: {assistant_output}")

Results from ChromaDB: {'ids': [['caracas_beach_(vieques)_-_wikipedia', 'domes_beach_-_wikipedia', 'cao_tiburones_-_wikipedia']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'category': 'Landmark', 'coordinates': "{'latitude': 18.108611111111113, 'longitude': -65.41305555555556}", 'description': 'Caracas Beach(Spanish:Playa Caracas), also known asRed BeachinEnglish, is a beach on the southern coast ofViequesin the barrio ofPuerto Ferro. The beach is famous for its white sand, its clear blue waters,tidal poolsand shallow cove perfect for snorkeling. It used to be considered remote but it is now easily reachable through a paved road. The beach has also been developed for visitors with picnic areas, bathrooms and a large parking area. It is located in and managed by theVieques National Wildlife Refugeand there is also a trail that leads to a nearby hill that offers beautiful views of the beach and cove.[1][2][3]  18631N652447W\ufeff /

AttributeError: 'list' object has no attribute 'get'